<center><img src="picture.jpg" width="600" height="500" /></center>

In [ ]:
import tensorflow as tf
import random
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.layers import concatenate, Input, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D  
from tensorflow import keras
from tensorflow.keras import backend as K 
import os
import shutil
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
# Define width, height, and channel variables
width, height, channel = 64, 64, 12

# Define the directory path for training data and list files in that directory
all_files_loc_train = 'E:/Deep Course/Weeks/W11/Data/Train(without Uncertainty)/'
all_files_train = os.listdir(all_files_loc_train)

# Define the directory path for testing data and list files in that directory
all_files_loc_test = 'E:/Deep Course/Weeks/W11/Data/Test(without Uncertainty)/'
all_files_test = os.listdir(all_files_loc_test)

# Create a dictionary to map input file names to corresponding label file names for training data
image_label_map = {
    "input_file_{}.npy".format(i + 1): "label_file_{}.npy".format(i + 1)
    for i in range(int(len(all_files_train) / 2))
}

# Create a list of training data file names that contain "input" in their names
partition_train = [item for item in all_files_train if "input" in item]

# Create a dictionary to map input file names to corresponding label file names for testing data
image_label_map_val = {
    "input_file_{}.npy".format(i + 1): "label_file_{}.npy".format(i + 1)
    for i in range(int(len(all_files_test) / 2))
}

# Create a list of testing data file names that contain "input" in their names
partition_val = [item for item in all_files_test if "input" in item]

# Print information about the lengths of data sets and partitions
print('Val Len:', len(all_files_test))
print('Len Val:', len(partition_val))
print('Train Len:', len(all_files_train))
print('Len Train:', len(partition_train))

In [ ]:
# Define a custom data generator class for training data
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, list_examples, batch_size=4, dim=(width, height, channel), shuffle=True):
        # Constructor of the data generator.
        self.dim = dim
        self.batch_size = batch_size
        self.list_examples = list_examples
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch
        return int(np.floor(len(self.list_examples) / self.batch_size))

    def __getitem__(self, index):
        # Generate one batch of data
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_examples[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        # This function is called at the end of each epoch.
        self.indexes = np.arange(len(self.list_examples))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        # Load individual numpy arrays and aggregate them into a batch.

        X = np.empty([self.batch_size, self.dim[0], self.dim[1], self.dim[2]], dtype='float32')

        # y is a one-hot encoded vector.
        y = np.empty([self.batch_size, width, height, 1], dtype=np.float32)

        # Generate data.

        c = 0
        for i in list_IDs_temp:

            x_file_path = os.path.join(all_files_loc_train, i)
            y_file_path = os.path.join(all_files_loc_train, image_label_map.get(i))

            # Load sample
            X[c, :, :, :] = np.load(x_file_path)
            # Load labels     
            y[c, :, :, :] = np.load(y_file_path)

            c = c + 1

        return X, y

# Define a custom data generator class for validation data (similar structure to the training data generator)
class ValDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, list_examples, batch_size=4, dim=(width, height, channel), shuffle=True):
        # Constructor of the data generator.
        self.dim = dim
        self.batch_size = batch_size
        self.list_examples = list_examples
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch
        return int(np.floor(len(self.list_examples) / self.batch_size))

    def __getitem__(self, index):
        # Generate one batch of data
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_examples[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        # This function is called at the end of each epoch.
        self.indexes = np.arange(len(self.list_examples))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        # Load individual numpy arrays and aggregate them into a batch.

        X = np.empty([self.batch_size, self.dim[0], self.dim[1], self.dim[2]], dtype=np.float32)

        # y is a one-hot encoded vector.
        y = np.empty([self.batch_size, width, height, 1], dtype=np.float32)

        # Generate data.

        c = 0
        for i in list_IDs_temp:

            x_file_path = os.path.join(all_files_loc_test, i)
            y_file_path = os.path.join(all_files_loc_test, image_label_map_val.get(i))

            # Load sample
            X[c, :, :, :] = np.load(x_file_path)
            # Load labels     
            y[c, :, :, :] = np.load(y_file_path)

            c = c + 1

        return X, y

# Create instances of the custom data generators for training and validation data
training_generator = DataGenerator(partition_train)
validation_generator = ValDataGenerator(partition_val)

In [ ]:
# Define constants ALPHA and BETA
ALPHA = 0.7
BETA = 0.3

# Define TverskyLoss function
def TverskyLoss(targets, inputs, alpha=ALPHA, beta=BETA, smooth=1e-6):
    """
    Tversky Loss function for semantic segmentation.
    :param targets: Ground truth labels
    :param inputs: Predicted labels
    :param alpha: Weight for false positives
    :param beta: Weight for false negatives
    :param smooth: Smoothing term to prevent division by zero
    :return: Tversky loss value
    """
    # Flatten label and prediction tensors
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)
    
    # Calculate True Positives, False Positives & False Negatives
    TP = K.sum((inputs * targets))
    FP = K.sum(((1 - targets) * inputs))
    FN = K.sum((targets * (1 - inputs)))
    
    # Calculate Tversky score
    Tversky = (TP + smooth) / (TP + alpha * FP + beta * FN + smooth)
    
    # Return Tversky loss
    return (1 - Tversky)

# Define Intersection over Union (IoU) function
def iou(y_true, y_pred, smooth=1):
    """
    Intersection over Union (IoU) metric for semantic segmentation.
    :param y_true: Ground truth labels
    :param y_pred: Predicted labels
    :param smooth: Smoothing term to prevent division by zero
    :return: IoU score
    """
    intersection = K.sum(y_true * y_pred)
    sum_ = K.sum(y_true + y_pred)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac

# Define Jaccard distance function
def jac_distance(y_true, y_pred):
    """
    Jaccard Distance (1 - IoU) metric for semantic segmentation.
    :param y_true: Ground truth labels
    :param y_pred: Predicted labels
    :return: Jaccard distance
    """
    y_truef = K.flatten(y_true)
    y_predf = K.flatten(y_pred)
    
    return -iou(y_true, y_pred)


In [ ]:

input_shape = (64, 64, 12)


learning_rate = 0.00001

batch_size = 2
num_epochs = 500
image_size = 64  # We'll resize input images to this size
patch_size = 8  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 128
num_heads = 8
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier



def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

def Ex_patches(images):
    patch_size = 8
    batch_size = tf.shape(images)[0]
    patches = tf.image.extract_patches(
        images=images,
        sizes=[1, patch_size, patch_size, 1],
        strides=[1, patch_size, patch_size, 1],
        rates=[1, 1, 1, 1],
        padding="VALID",
    )
    patch_dims = patches.shape[-1]
    
    patches = tf.reshape(patches, [batch_size, -1, patch_dims])
    
    return patches


def patch_encoder(patch, num_patches, projection_dim):
    positions = tf.range(start=0, limit=num_patches, delta=1)
    
    emd=layers.Embedding(input_dim=num_patches, output_dim=projection_dim)(positions)
    dens=layers.Dense(units=projection_dim)(patch)
    
    encoded = emd + dens

    return encoded



In [ ]:
def create_vit_classifier(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    
    # Create patches.
    patches = Ex_patches(inputs)
    # print(patches.shape)
    # Encode patches.
    encoded_patches = patch_encoder(patches, num_patches, projection_dim)

    # Create multiple layers of the Transformer block.
    
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
#         print(x1.shape)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        
#         print(attention_output.shape)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        print(x3.shape)
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # print(x3.shape)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])
#         print(encoded_patches.shape)

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.1)

    logits=layers.Dense(units=64*64,activation='sigmoid')(features)
    
    out=layers.Reshape((64,64,1))(logits)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=out)
    return model

In [ ]:
model = create_vit_classifier((64,64,12))

In [ ]:
model = create_vit_classifier((64,64,12))

# Define the learning rate for the optimizer
learning_rate = 0.0005

# Define the optimizer (RMSprop in this case) with the specified learning rate
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

# Compile the model with a custom loss function (TverskyLoss) and metrics (IoU)
model.compile(loss=TverskyLoss, optimizer=optimizer, metrics=[iou])

# Define an early stopping callback to monitor validation loss and stop training if it doesn't improve for a certain number of epochs
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=60)

# Create a list of callbacks, including ModelCheckpoint to save the best model and EarlyStopping for early stopping
callbacks = [ModelCheckpoint('E:/Deep Course/Weeks/W11/Models/ViT.h5',
                             verbose=1, save_best_only=True), early_stopping]


In [ ]:
history =model.fit(training_generator,epochs=500,validation_data=validation_generator,callbacks=callbacks)